In [1]:
import numpy as np

In [2]:
pb = 1000.
pb_half = 975.
pt = 100.
pt_half = 125.

In [3]:
g=9.8066
kappa=0.2856219
R=287.05
p0=1.e5
alpha=-1./5.255877
cp = 1004.7

In [4]:
p = np.linspace(pb,pt,19)
p_half = np.linspace(pb_half,pt_half,18)

Code below defines standard atmosphere geopotential, temperature, and stratification (on half-levels)

In [5]:
def standard_atmosphere(p, p_half):
    g=9.8066
    kappa=0.2856219
    alpha=-1./5.255877
    beta=-6341.624
    gamma=.0065
    R=287.04
    cp = 1004.7
    p00=1.e5
    nlevels = p.size
    nlevels_half = p_half.size

    Z = np.zeros(nlevels)
    phi = np.zeros(nlevels)
    T = np.zeros(nlevels)
    theta = np.zeros(nlevels)

    T_half = np.zeros(nlevels)
    theta_half = np.zeros(nlevels_half)
    dthetadp = np.zeros(nlevels_half)
    S = np.zeros(nlevels_half)

    for k in np.arange(nlevels):
        if (p[k] > 226.32):
            Z[k] = g*(288.15/gamma) * ( 1. - (1013.25/p[k])**alpha)
            T[k] = 288.15 - 0.0065*Z[k]
            theta[k] = T[k] * (1000./p[k])**(R/cp)
        else:
            T[k] = 216.65
            Z[k] = g*(11.e3+beta*np.log(p[k]/226.32))
            theta[k] = T[k] * (1000./p[k])**(R/cp)
                
    for k in np.arange(nlevels_half):
        if (p_half[k] > 226.32):
            T_half[k] = 288.15 - 0.0065*Z[k]
            theta_half[k] = T_half[k] * (1000./p_half[k])**(R/cp)
        else:
            T_half[k] = 216.65
            theta_half[k] = T_half[k] * (1000./p_half[k])**(R/cp)
    
    for k in np.arange(nlevels_half):
        dthetadp[k] = (theta[k+1] - theta[k-1])/(1.e2*(p[k+1]-p[k-1]))
        S[k] =  -R*(T_half[k]/theta_half[k])*dthetadp[k]/(p[k]*100.)
    return g*Z, T, S

In [6]:
print( standard_atmosphere(p, p_half))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
(array([  10663.6950352 ,   51963.92436738,   95063.51330908,
        140147.69281393,  187433.09532096,  237175.43933892,
        289679.75238627,  345314.22034486,  404529.34187002,
        467885.05685608,  536090.24260362,  610062.10956684,
        691019.04255285,  780632.69656955,  881292.13787904,
        996598.13788779, 1133263.25323866, 1308711.97563434,
       1555993.06126933]), array([ 281.0819012 ,  253.70732788,  225.14010498,  195.25745791,
        163.91579859,  130.94563195,   96.14477999,   59.26920215,
         20.02029937,  -21.97306707,  -67.18075448, -116.2107073 ,
       -169.87049401, -229.2681192 , -295.98710116, -372.41409931,
        216.65      ,  216.65      ,  216.65      ]), array([ 5.39154268e-06, -1.44973589e-05, -1.62447665e-05, -1.83196229e-05,
       -2.08081645e-05, -2.38268142e-05, -2.75357574e-05, -3.21601965e-05,
       -3.80246711e-05, -4.56104878e-05, -5.56558982e-05, -6.93396456e-05,
       -8.86377